In [78]:
import uuid
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [17]:
from bs4 import BeautifulSoup

def get_urls(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    form = soup.find('form', action='meetsearch.php')
    select = form.find('select', attrs={'name':'c'})
    options = select.find_all('option')

    urls = ["https://www.usms.org/comp/meets/meetsearch.php?c={}&MeetID=20230805UMBC50L".format(option['value']) for option in options]

    return urls

In [18]:
import requests
url = 'https://www.usms.org/comp/meets/meet.php?MeetID=20230805UMBC50L'
#url = 'https://www.usms.org/comp/meets/meetsearch.php'
response = requests.get(url)

if response.status_code == 200:
    urls = get_urls(response.text)
    print(urls)
else:
    print("Failed to retrieve the webpage")
    

['https://www.usms.org/comp/meets/meetsearch.php?c=1129628&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129593&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129570&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129553&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129564&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129617&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129594&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129632&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129595&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129578&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129591&MeetID=20230805UMBC50L', 'https://www.usms.org/comp/meets/meetsearch.php?c=1129538&MeetID=20230805UMBC50L', 'ht

In [24]:
response = requests.get(urls[10])

In [25]:
response.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"\r\n    "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n\r\n<script type=\'text/javascript\'>\r\n/* <![CDATA[ */\r\nvar googletag = googletag || {};\r\ngoogletag.cmd = googletag.cmd || [];\r\n(function() {\r\nvar gads = document.createElement(\'script\');\r\ngads.async = true;\r\ngads.type = \'text/javascript\';\r\nvar useSSL = \'https:\' == document.location.protocol;\r\ngads.src = (useSSL ? \'https:\' : \'http:\') + \r\n\'//www.googletagservices.com/tag/js/gpt.js\';\r\nvar node = document.getElementsByTagName(\'script\')[0];\r\nnode.parentNode.insertBefore(gads, node);\r\n})();\r\n/* ]]> */\r\n</script>\r\n\r\n<script type=\'text/javascript\'>\r\n/* <![CDATA[ */\r\n\tgoogletag.cmd.push(function() {\r\n\t\r\n\t var mappingLeaderboard8 =\r\n                    googletag.sizeMapping().\r\n                    addSize([1200, 200], [940, 118]).\r\n      

In [27]:
import re
import pandas as pd
def extract_results(html):
    # Creating a Beautiful Soup object
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find the preformatted text containing the results
    results_pre = soup.find('pre')
    
    # Extract the text and split by lines
    results_text = results_pre.get_text()
    lines = results_text.split("\n")
    
    # Initialize the variables to hold the data
    data = []
    current_event_name = None
    
    # Iterate through the lines to extract data
    for line in lines:
        if "Meter" in line:
            # Event name detected, update current event name
            current_event_name = line.strip().replace("Men", "").strip()
        elif "===" in line or not line.strip():
            # Skip header or empty lines
            continue
        else:
            # Extract information using regular expressions
            match = re.match(r'\s*\d+\s+(?P<name>[\w\s\,]+)\s+(?P<age>\d+)\s+(?P<club>\w+)\s+(?P<seed_time>[\d\:\.]+)\s+(?P<final_time>[\d\:\.]+)', line)
            if match:
                name = match.group('name').strip()
                age = int(match.group('age'))
                club = match.group('club')
                seed_time = match.group('seed_time')
                final_time = match.group('final_time')
                row = [name, age, current_event_name, club, seed_time, final_time]
                data.append(row)
    
    # Creating a DataFrame using the extracted data
    columns = ["name", "age", "event name", "club", "seed time", "final time"]
    df = pd.DataFrame(data, columns=columns)
    
    return df

In [29]:
response = requests.get(urls[16])
extract_results(response.text)

,name,age,event name,club,seed time,final time
0,"Corlette, Sabrina",51,Women 50-54 100 Meter Freestyle Finals,ALEX,1:10.75,1:12.63
1,"Corlette, Sabrina",51,Women 50-54 50 Meter Butterfly Finals,ALEX,36.69,37.67


In [30]:
def extract_results2(html):
    # Create a BeautifulSoup object
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the preformatted text containing the results
    results_pre = soup.find('pre', {'style': 'white-space: pre; font-size:13px; font-family: monospace'})

    # Get the results text
    results_text = results_pre.text

    # Split the results into lines
    lines = results_text.split('\n')

    # Define the columns
    columns = ["name", "age", "event name", "club", "seed time", "final time", "gender"]

    # Create an empty list to store the rows
    rows = []

    # Initialize variables for storing the gender and event name
    gender = None
    event_name = None

    # Iterate through the lines and parse the results
    for line in lines:
        # Check if the line contains gender information (Men's Results or Women's Results)
        if "Men's Results" in line:
            gender = 'Men'
        elif "Women's Results" in line:
            gender = 'Women'

        # Check if the line contains an event name
        if 'Meter' in line:
            event_name = line

        # Use regular expressions to match the line with the results pattern
        result_match = re.match(r'\s*\d+\s+(?P<name>[\w\s,]+)\s+(?P<age>\d+)\s+(?P<club>\w+)\s+(?P<seed_time>[\d\.:]+)\s+(?P<final_time>[\d\.:]+)', line)
        if result_match:
            # Extract the results
            name = result_match.group('name').strip()
            age = result_match.group('age')
            club = result_match.group('club')
            seed_time = result_match.group('seed_time')
            final_time = result_match.group('final_time')

            # Append the row to the rows list
            rows.append([name, age, event_name, club, seed_time, final_time, gender])

    # Create a pandas DataFrame using the rows and columns
    df = pd.DataFrame(rows, columns=columns)

    return df

In [31]:
response = requests.get(urls[16])
extract_results2(response.text)

,name,age,event name,club,seed time,final time,gender
0,"Corlette, Sabrina",51,Women 50-54 100 Meter Freestyle Finals,ALEX,1:10.75,1:12.63,Women
1,"Corlette, Sabrina",51,Women 50-54 50 Meter Butterfly Finals,ALEX,36.69,37.67,Women


In [70]:
def extract_results3(html, verbose = False):
    soup = BeautifulSoup(html, 'html.parser')
    results_pre = soup.find('pre', {'style': 'white-space: pre; font-size:13px; font-family: monospace'})
    results_text = results_pre.text
    lines = results_text.split('\n')
    columns = ["name", "age", "event name", "club", "seed time", "final time", "gender", "stroke type", "distance", "unit"]
    rows = []
    gender = None
    event_name = None
    stroke_type = None
    distance = None
    unit = None

    for line in lines:
        if verbose:
            print("line")
            print(line)
        if "Men's Results" in line:
            gender = 'Men'
        elif "Women's Results" in line:
            gender = 'Women'

        if 'Meter' in line or 'Yard' in line:
            event_name = line
            #stroke_match = re.search(r'(?P<distance>\d+)\s+(Meter|Yard)\s+(?P<stroke_type>\w+)\s+', line)
            stroke_match = re.search(r'(?P<distance>\d+)\s+(Meter|Yard)\s+(?P<stroke_type>[\w\s]+)\s+', line)

            if stroke_match:
                if verbose:
                    print("stroke_match")
                    print(stroke_match)
                distance = stroke_match.group('distance')
                unit = 'Meters' if 'Meter' in line else 'Yards'
                stroke_type = stroke_match.group('stroke_type')


        result_match = re.match(r'\s*\d+\s+(?P<name>[\w\s,]+)\s+(?P<age>\d+)\s+(?P<club>\w+)\s+(?P<seed_time>[\d\.:]+)\s+(?P<final_time>[\d\.:]+)', line)
        if result_match:
            if verbose:
                print("result_match")
                print(result_match)
            name = result_match.group('name').strip()
            age = result_match.group('age')
            club = result_match.group('club')
            seed_time = result_match.group('seed_time')
            final_time = result_match.group('final_time')
            rows.append([name, age, event_name, club, seed_time, final_time, gender, stroke_type, distance, unit])

    df = pd.DataFrame(rows, columns=columns)
    return df

In [72]:
extract_results3(requests.get(urls[6]).text)

,name,age,event name,club,seed time,final time,gender,stroke type,distance,unit
0,"Bergamini, Margaret J",60,Women 60-64 50 Meter Freestyle Finals,MARY,36.32,36.05,Women,Freestyle,50,Meters
1,"Bergamini, Margaret J",60,Women 60-64 200 Meter Backstroke Finals,MARY,3:15.50,3:24.33,Women,Backstroke,200,Meters
2,"Bergamini, Margaret J",60,Women 60-64 50 Meter Butterfly Finals,MARY,41.77,44.07,Women,Butterfly,50,Meters
3,"Bergamini, Margaret J",60,Women 60-64 200 Meter Butterfly Finals,MARY,3:32.50,3:55.52,Women,Butterfly,200,Meters


In [56]:
dfs = []

for a_url in urls:
    response = requests.get(a_url)
    dfs.append(extract_results3(response.text))

In [62]:
bigdf = pd.concat(dfs)

In [63]:
bigdf = bigdf.reset_index(drop=True)

In [64]:
bigdf.head(20)

,name,age,event name,club,seed time,final time,gender,stroke type,distance,unit
0,"Arena, Christopher J",24,Men 18-24 100 Meter Freestyle Finals,DCAC,59.00,59.04,Men,Freestyle,100,Meters
1,"Arena, Christopher J",24,Men 18-24 100 Meter Butterfly Finals,DCAC,1:05.00,1:02.85,Men,Butterfly,100,Meters
2,"Arena, Christopher J",24,Men 18-24 200 Meter Individual Medley Finals,DCAC,2:25.00,2:21.97,Men,Individual Medley,200,Meters
3,"Bell Vanderhoof, Yvonne P",75,Women 75-79 100 Meter Freestyle Finals,DCWW,3:37.16,3:36.13,Women,Freestyle,100,Meters
4,"Bell Vanderhoof, Yvonne P",75,Women 75-79 200 Meter Breaststroke Finals,DCWW,8:50.50,8:34.83,Women,Breaststroke,200,Meters
5,"Bennett, Ruth R",77,Women 75-79 50 Meter Freestyle Finals,RMST,41.47,44.79,Women,Freestyle,50,Meters
6,"Bennett, Ruth R",77,Women 75-79 100 Meter Freestyle Finals,RMST,1:37.08,1:38.02,Women,Freestyle,100,Meters
7,"Bennett, Ruth R",77,Women 75-79 200 Meter Freestyle Finals,RMST,3:25.22,3:36.68,Women,Freestyle,200,Meters
8,"Bergamini, Margaret J",60,Women 60-64 50 Meter Freestyle Finals,MARY,36.32,36.05,Women,Freestyle,50,Meters
9,"Bergamini, Margaret J",60,Women 60-64 200 Meter Backstroke Finals,MARY,3:15.50,3:24.33,Women,Backstroke,200,Meters


In [65]:
bigdf.age.value_counts()

age
42    15
34    13
61    11
33    10
32    10
74     9
43     8
25     8
36     8
78     7
26     6
71     6
60     6
41     5
56     5
40     5
68     4
28     4
24     4
54     4
58     4
57     4
20     4
19     4
69     4
49     4
73     4
67     4
27     3
75     3
85     3
87     3
77     3
23     3
37     3
50     3
21     3
65     2
35     2
31     2
51     2
46     2
70     2
38     2
39     2
66     1
62     1
59     1
Name: count, dtype: int64

In [66]:
bigdf.gender.value_counts()

gender
Men      127
Women     99
Name: count, dtype: int64

In [94]:

def extract_meet_data(html):
    soup = BeautifulSoup(html, 'html.parser')

    rows = soup.select('table.meetlist tr')
    data = []

    for row in rows[1:]:
        date = row.select('td')[0].text.strip()
        course = row.select('td')[1].text.strip()
        lmsc = row.select('td')[2].text.strip()
        meet_name = row.select('td a')[0].text.strip()
        meet_url = f"https://www.usms.org/comp/meets/{row.select('td a')[0]['href']}"
        meet_id = re.search(r'MeetID=(\d+)', meet_url).group(1) if re.search(r'MeetID=(\d+)', meet_url) else None
        random_uuid = str(uuid.uuid4())
        
        data.append([date, course, lmsc, meet_name, meet_url, random_uuid, meet_id])

    columns = ["date", "course", "LMSC", "Meet Name", "meet URL", "UUID"]
    df = pd.DataFrame(data, columns=columns)

    return df

In [80]:
meet_url = "https://www.usms.org/comp/meets/meetlist.php"
meet_response = requests.get(meet_url)

In [81]:
df, next_url = extract_meet_data(meet_response.text)

In [82]:
df.head()

,date,course,LMSC,Meet Name,meet URL,UUID
0,"Aug 5, 2023",LCM,MD,Summer Splash 2023,http://www.usms.orgmeet.php?MeetID=20230805UMB...,0fba7530-2b12-45f0-940a-9c8cf89ac651
1,"Aug 5, 2023",LCM,OR,2023 Oregon Masters LCM Association and NW Zon...,http://www.usms.orgmeet.php?MeetID=2023080523O...,c623a56a-5a66-4891-9f6c-02f0f3d863df
2,"Aug 2-6, 2023",LCM,FL,2023 U.S. Masters Swimming Summer National Cha...,http://www.usms.orgmeet.php?MeetID=20230802SMR...,a3eba548-f9d7-4089-b30f-b56a003c8614
3,"Jul 30, 2023",LCM,AR,Old School Spirit - Long Course Meters Meet (O...,http://www.usms.orgmeet.php?MeetID=20230730OSS23L,9559a2aa-49c8-465f-9980-1854ffd19112
4,"Jul 29-30, 2023",LCM,KY,46th Annual Lakeside Masters Long Course Invit...,http://www.usms.orgmeet.php?MeetID=20230729Lak...,0d9fb413-751a-416a-971b-8e7071a34f45


In [84]:
print(next_url)

None


In [89]:
def meet_url_generator(page = 0):
    return  f"https://www.usms.org/comp/meets/meetlist.php?page={page}&CourseID=0&LMSCID="

In [90]:
meet_urls = [meet_url_generator(100*x) for x in range(0,20)]

In [91]:
meet_response = requests.get(meet_urls[0])

In [96]:
meet_df = extract_meet_data(meet_response.text)

In [97]:
meet_df.head()

,date,course,LMSC,Meet Name,meet URL,UUID
0,"Aug 5, 2023",LCM,MD,Summer Splash 2023,https://www.usms.org/comp/meets/meet.php?MeetI...,1e0feeb7-d67c-410e-99bd-7e13f7683c15
1,"Aug 5, 2023",LCM,OR,2023 Oregon Masters LCM Association and NW Zon...,https://www.usms.org/comp/meets/meet.php?MeetI...,1b97b7e4-0a67-40b6-aecc-cba9c0a9f8a4
2,"Aug 2-6, 2023",LCM,FL,2023 U.S. Masters Swimming Summer National Cha...,https://www.usms.org/comp/meets/meet.php?MeetI...,17fbf447-1c58-42e9-b707-762afff8803f
3,"Jul 30, 2023",LCM,AR,Old School Spirit - Long Course Meters Meet (O...,https://www.usms.org/comp/meets/meet.php?MeetI...,44f95f16-451e-4f7e-bd31-c099972484b8
4,"Jul 29-30, 2023",LCM,KY,46th Annual Lakeside Masters Long Course Invit...,https://www.usms.org/comp/meets/meet.php?MeetI...,94723549-5556-496b-a6fe-08fb989fd904


In [101]:
a_url = meet_df[["meet URL","UUID"]]

In [108]:
a_url.to_numpy()

AttributeError: 'DataFrame' object has no attribute 'aslist'

In [109]:
bigdf.shape

(226, 10)